# 📽️ Subtitle Extractor & Summarizer Project
extracts burned-in subtitles from a video and summarizes them using an LLM.

In [ ]:
# Install dependencies
!pip install opencv-python pytesseract transformers openai

In [ ]:
# Step 1: Extract subtitle frames using OpenCV
import cv2, os

def extract_subtitle_frames(video_path, output_dir, step=10, crop_height_ratio=0.2):
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    saved = 0

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_id % step == 0:
            h, w, _ = frame.shape
            cropped = frame[int(h * (1 - crop_height_ratio)):, :]
            cv2.imwrite(f"{output_dir}/frame_{saved}.png", cropped)
            saved += 1
        frame_id += 1
    cap.release()

In [ ]:
# Step 2: Extract text from subtitle frames using OCR
from PIL import Image
import pytesseract

def extract_text_from_images(images_folder):
    texts = []
    for file in sorted(os.listdir(images_folder)):
        if file.endswith(".png"):
            img_path = os.path.join(images_folder, file)
            img = Image.open(img_path)
            text = pytesseract.image_to_string(img, lang='eng')
            if text.strip():
                texts.append(text.strip())
    return texts

In [ ]:
# Step 3: Clean duplicate subtitles
def clean_subtitles(text_list):
    cleaned = []
    prev = ""
    for line in text_list:
        if line != prev:
            cleaned.append(line)
            prev = line
    return cleaned

In [ ]:
# Step 4: Summarize subtitles using HuggingFace Transformers
from transformers import pipeline

def summarize_text(texts, max_tokens=512):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    full_text = " ".join(texts)
    chunks = [full_text[i:i+max_tokens] for i in range(0, len(full_text), max_tokens)]
    summaries = [summarizer(chunk)[0]['summary_text'] for chunk in chunks]
    return " ".join(summaries)

In [ ]:
!pip install yt-dlp
!yt-dlp https://www.youtube.com/watch?v=YInY5GSey-Q -o sample_hardsub.mp4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.7 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=YInY5GSey-Q
[youtube] YInY5GSey-Q: Downloading webpage
[youtube] YInY5GSey-Q: Downloading tv client config
[youtube] YInY5GSey-Q: Downloading player a10d7fcc-main
[youtube] YInY5GSey-Q: Downloading tv player API JSON
[youtube] YInY5GSey-Q: Downloading ios player API JSON
[youtube] YInY5GSey-Q: Downloading m3u8 information
[info] YInY5GSey-Q: Downloading 1 format(s): 137+251
[download] Destination: sample_hardsub.mp4.f137.mp4
[download] 100% of   22.82MiB in 00:00:01 at 14.81MiB/s
[download] Destination: sample_hardsub.mp4.f251.webm
[download] 100% of    5.87MiB in 00:00:00 at 63.89MiB/s
[Merger] Merging formats into "sample_hardsub.mp4.mkv"
Deleting original file sample_hardsub.mp4.f137.mp4 (pass -k to keep)
Deleting original file sample_hardsub.mp4.f251.webm (pass -k to ke

In [ ]:
# Step 5: Run all steps together (example)
video_path = "/content/sample_hardsub.mp4.mkv"
output_dir = "frames"

extract_subtitle_frames(video_path, output_dir)
raw_texts = extract_text_from_images(output_dir)
cleaned = clean_subtitles(raw_texts)
summary = summarize_text(cleaned)

print("\n📄 Summary:\n", summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 142, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)



📄 Summary:
 V1.1.0 enables standalone version of PaddleOCR from htt. The GPU version requires an Nvidia graphics card and driver version 522.25 or higher. No further changes to CLI versions since v1.2.0. For more information on Paddle OCR, visit the official website. Everything can now be done via a few clicks instead of needing to tinker with the command line. with an easy-to-use graphical user interface. With the latest version of VideOCR, you can now use the following commands: VideOCR-CPU-v1.1/Vide OCR-setup-x64.exe. This new release comes with an easy-to-use GUI. Everything can now be done via a few clicks instead of needing to tinker with the command line!https://github.com/timminator/VideOCR/releases/download/v1.1.0.1/Vi.  deOCR-CPU-v1.1.0.1 is now available for Windows 8 and Windows 7. This new release comes with an easy-to-use GUI. Everything can now be done via a few clicks instead of needing to tinker with the command line! VideOCR-CPU-v1.1.0.0 comes with an easy-to-use GUI

In [8]:
# Step 6: Gradio Interface to upload video and get summary
import gradio as gr

def process_video_and_summarize(video_file):
    import tempfile, os
    video_path = video_file.name
    temp_dir = tempfile.mkdtemp()

    extract_subtitle_frames(video_path, temp_dir)
    raw_texts = extract_text_from_images(temp_dir)
    cleaned = clean_subtitles(raw_texts)
    if not cleaned:
        return "⛔ No subtitle text detected."
    summary = summarize_text(cleaned)
    return summary

gr.Interface(
    fn=process_video_and_summarize,
    inputs=gr.Video(label="Upload video with burned-in subtitles"),
    outputs="text",
    title="🎬 Subtitle Summarizer",
    description="Upload a video with hardcoded subtitles. This app will extract subtitles using OCR and summarize them using an LLM."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a76a589905cedae323.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
